In [439]:
import urllib
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import umap
import itertools

from maayanlab_bioinformatics.enrichment import enrich_crisp

from sklearn.decomposition import NMF

# Bokeh
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, CustomJS, ColumnDataSource, Slider, Range1d
from bokeh.layouts import column
from bokeh.palettes import all_palettes
output_notebook()

Loading BokehJS ...


    ,
    ,
    'DSigDB',
    'GeneSigDB',
    'GWAS_Catalog_2019',
    'LINCS_L1000_Chem_Pert_down',
    'LINCS_L1000_Chem_Pert_up',
    'LINCS_L1000_Ligand_Perturbations_down',
    'LINCS_L1000_Ligand_Perturbations_up',
    'MSigDB_Computational',
    'MSigDB_Oncogenic_Signatures',
    'Old_CMAP_down',
    'Old_CMAP_up',
    'OMIM_Disease',
    'OMIM_Expanded',
    'PheWeb_2019',
    'Rare_Diseases_AutoRIF_ARCHS4_Predictions',
    'Rare_Diseases_AutoRIF_Gene_Lists',
    'Rare_Diseases_GeneRIF_ARCHS4_Predictions',
    'Rare_Diseases_GeneRIF_Gene_Lists',
    'UK_Biobank_GWAS_v1',
    'Virus_Perturbations_from_GEO_down',
    'Virus_Perturbations_from_GEO_up',
    'VirusMINT'
    ## NOT VIRUS HOST

    # take a closer look at:
    'GO_Biological_Process_2018'
    ['MGI_Mammalian_Phenotype_Level_4_2019']
    'DisGeNET'

In [440]:
all_libraries = ['DrugMatrix']
#genes = ['TP53', 'TNF', 'EGFR', 'GKN1', 'HADHA', 'APOE', 'ESR1', 'VEGFA', 'TGFB1', 'PREPL', 'TIA1', 'TPO', 'TTN', 'SATB2', 'CHPF', 'MALL', 'MIPIP', 'NUPL1', 'IL6', 'PDIA3', 'CTNNB1', 'SLC39A1', 'DTNA','SLC1A1', 'GALNT2', 'HIST2H2AC', 'CD63']

# open_gene_list_file = open('geneList.txt','r')
# lines = open_gene_list_file.readlines()
# genes = [x.strip().upper() for x in lines]
# open_gene_list_file.close()

significance_value = 0.05

In [441]:
# open Enrichr library from online
def get_Enrichr_library(library_index):
    # processes library data
    raw_library_data = []
    library_data = []

    with urllib.request.urlopen('https://amp.pharm.mssm.edu/Enrichr/geneSetLibrary?mode=text&libraryName=' + all_libraries[library_index]) as f:
        for line in f.readlines():
                raw_library_data.append(line.decode("utf-8").split("\t\t"))

    name = []
    gene_list = []

    for i in range(len(raw_library_data)):
        name += [raw_library_data[i][0]]
        raw_genes = raw_library_data[i][1].replace('\t', ' ')
        gene_list += [raw_genes[:-1]]

    library_data = [list(a) for a in zip(name, gene_list)]
    
    return library_data

In [442]:
library_data = get_Enrichr_library(0)

df = pd.DataFrame(data = library_data, columns = ['Name', 'Genes'])

gene_list = df['Genes']

tfidf_vectorizer = TfidfVectorizer(
    min_df=1,
    max_df=0.85,
    max_features = 10000,
    ngram_range=(1, 1)
)

tfidf = tfidf_vectorizer.fit_transform(gene_list)

# Save the feature names for later to create topic summaries
tfidf_fn = tfidf_vectorizer.get_feature_names()

# plot after tfidf

reduce = umap.UMAP()
reduce.fit(tfidf)
embedding = reduce.transform(tfidf)

embedding = pd.DataFrame(embedding, columns=['x','y'])

source1 = ColumnDataSource(
        data=dict(
            x = embedding.x,
            y = embedding.y,
            alpha = [0.7] * embedding.shape[0],
            size = [7] * embedding.shape[0],
            gene_set = df['Name']
        )
    )

print(embedding.shape[0])

hover_emb = HoverTool(names=["df"], tooltips="""
    <div style="margin: 10">
        <div style="margin: 0 auto; width:300px;">
            <span style="font-size: 12px; font-weight: bold;">Gene Set:</span>
            <span style="font-size: 12px">@gene_set</span>
        </div>
    </div>
    """)
tools_emb = [hover_emb, 'pan', 'wheel_zoom', 'reset']
plot_emb = figure(plot_width=700, plot_height=700, tools=tools_emb)
plot_emb.circle('x', 'y', size='size', 
                 alpha='alpha', line_alpha=0, line_width=0.01, source=source1, name="df")

show(plot_emb)

TypeError: a bytes-like object is required, not 'list'

In [430]:
# create and run NMF model

n_comp = 10

nmf = NMF(
    n_components=n_comp, 
    max_iter=100000,
    alpha=0.0
)

W = nmf.fit_transform(tfidf)
H = nmf.components_
nmf_embedding = nmf.transform(tfidf)

In [431]:
# enrichment analysis
def get_library_iter(library_data):
    for member in library_data:
        term = member[0]
        gene_set = member[1].split(' ')
        yield term, gene_set

def get_enrichment_results(genes, library_data):
    return sorted(enrich_crisp(genes, get_library_iter(library_data), 20000, True), key=lambda r: r[1].pvalue)

def get_pvalue(row, unzipped_results, all_results):
    if row['Name'] in list(unzipped_results[0]):
        index = list(unzipped_results[0]).index(row['Name'])
        return all_results[index][1].pvalue
    else:
        return 1

In [432]:
# call UMAP

reducer = umap.UMAP()
reducer.fit(W)
embedding = reducer.transform(W)

embedding = umap.UMAP().fit_transform(tfidf.todense())
embedding = pd.DataFrame(embedding, columns=['x','y'])

# combine embedding with df
df = pd.concat([embedding, df], axis=1)
df.to_csv('Libraries/' + all_libraries[0] + '.csv', index = False)

In [433]:
# # call enrichment results
# all_results = get_enrichment_results(genes, library_data)
# unzipped_results = list(zip(*all_results))

# # add p value to the dataframe
# df['p value'] = df.apply (lambda row: get_pvalue(row, unzipped_results, all_results), axis=1)

# my_colors = []
# for index, row in df.iterrows():
#     if row['p value'] < significance_value:
#         my_colors += ['#000000']
#     else:
#         my_colors += [all_palettes['Category20'][20][0]]

embedding['hue'] = nmf_embedding.argmax(axis = 1)
my_colors = [all_palettes['Category20'][20][i] for i in embedding.hue]

source = ColumnDataSource(
        data=dict(
            x = embedding.x,
            y = embedding.y,
            gene_set = df['Name'],
            #p_value = df['p value'],
            colors = my_colors
        )
    )

hover_emb = HoverTool(names=["df"], tooltips="""
    <div style="margin: 10">
        <div style="margin: 0 auto; width:300px;">
            <span style="font-size: 12px; font-weight: bold;">Gene Set:</span>
            <span style="font-size: 12px">@gene_set</span>
            <span style="font-size: 12px; font-weight: bold;">p-value:</span>
            <span style="font-size: 12px">@p_value</span>
        </div>
    </div>
    """)
tools_emb = [hover_emb, 'pan', 'wheel_zoom', 'reset']
plot_emb = figure(plot_width=700, plot_height=700, tools=tools_emb)
plot_emb.circle('x', 'y', size = 7, alpha = 0.7, line_alpha = 0, 
                line_width = 0.01, source = source, fill_color = 'colors', name = "df")

show(plot_emb)

In [434]:
# this will print out the most common genes in each group
# not super useful but kind of interesting
n_topics = n_comp
n_top_words = 15

print("Topics found via NMF:")
for topic_idx, topic in enumerate(nmf.components_):
    print("\nTopic {}:".format(topic_idx+1))
    print(" ".join(['[{}]'.format(tfidf_fn[i]) for i in topic.argsort()[:-n_top_words - 1:-1]]))
print()

Topics found via NMF:

Topic 1:
[edf1] [atp6v0e1] [uba3] [aifm1] [tyms] [stambp] [efr3a] [rab6a] [egfr] [arhgef7] [eif4e2] [ufc1] [adar] [tfdp1] [shq1]

Topic 2:
[ugt1a5] [ppp3cb] [ifna4] [map2k4] [phgr1] [c14orf2] [iqcb1] [rgs13] [fam133a] [slc25a40] [rcor1] [cox19] [tmem257] [ifna5] [ankmy2]

Topic 3:
[mrpl38] [mrps34] [mtg2] [mrps25] [ndufv1] [mrpl55] [mrpl44] [ndufa13] [ndufs8] [mrps27] [mrpl46] [cox5a] [gfm1] [mrpl21] [ndufa3]

Topic 4:
[tceal7] [hmgn5] [gype] [kras] [s100g] [skap2] [prdx3] [znf429] [hist1h2bd] [minos1] [yeats4] [defb125] [wfdc9] [angptl3] [ctnnb1]

Topic 5:
[scarf1] [maged4] [krtap4] [kcnj9] [snapin] [drd5] [extl3] [scaf1] [celsr2] [eif1ay] [ypel5] [psmd8] [ccdc174] [tceal3] [ubc]

Topic 6:
[isl1] [mycn] [pdcd6ip] [irs2] [fam175a] [ldb1] [furin] [cit] [c15orf41] [eif3k] [bcl2] [igf1r] [suz12] [hand2] [fam104b]

Topic 7:
[fpgs] [mthfd1] [nampt] [rfk] [dhodh] [atp1b3] [umps] [tada2b] [pfas] [cad] [gfpt1] [nmt1] [mvd] [med23] [ippk]

Topic 8:
[srf] [adat3] [c19orf52